In [23]:
from py2neo import Graph, Node, Relationship, NodeMatcher
import pandas as pd
graph = Graph('http://localhost:7474/db/data/', auth=("neo4j","123"))

In [24]:
stock = pd.read_csv('1company_data_table.csv', encoding='gbk')
# print(stock)
# 空值填充为未知
stock['行业'] = stock['行业'].fillna('未知')
# 去重
stock = stock.drop_duplicates(subset=None, keep='first', inplace=False)
# 创建股票信息实体
for i in stock.values:
    a = Node('股票', TS代码=i[0], 股票名称=i[1], 公司名称=i[2], 公司成立时间=i[5],
                行业=i[6], 公司介绍=i[7], 公司联系电话=i[-1])
    graph.create(a)

In [25]:
# 创建股东实体
holder = pd.read_csv('1sharehold_data_table.csv', encoding='gbk')
# print(holder)
holder_iden = holder[['股东名称', '股东性质']].drop_duplicates(keep='first', inplace=False)
for i in holder_iden.values:
    a = Node('股东',  股东名称=i[0],  股东性质=i[1])
    graph.create(a)

In [26]:
# 创建参股关系
matcher = NodeMatcher(graph)
for i in holder.values:
    a = matcher.match("股票", TS代码=i[0]).first()
    b = matcher.match("股东", 股东名称=i[1]).first()
    r = Relationship(b, '参股', a , 股份类型=i[3], 持股数=i[4], 占总股本比例=i[5])
    graph.create(r)

In [27]:
fund = pd.read_csv('1fund_data_table.csv', encoding='gbk')
fund_iden = fund[['基金名称', '基金代码']].drop_duplicates(keep='first', inplace=False)
for i in fund_iden.values:
    a = Node('基金', 基金名称=i[0], 基金代码=i[1])
    graph.create(a)
matcher = NodeMatcher(graph)
for i in fund.values:
    a = matcher.match('股票', TS代码=i[0]).first()
    b = matcher.match('基金', 基金名称=i[2]).first()
    r = Relationship(b, '持股', a, 持股数=i[3], 占总股本比例=i[4])
    graph.create(r)

In [28]:
concept = pd.read_csv('1concept_data_table.csv', encoding='gbk')

concept_iden = concept[['概念']].drop_duplicates(keep='first', inplace=False)
for i in concept_iden.values:
    a = Node('概念', 概念名称=i[0])
    graph.create(a)

matcher = NodeMatcher(graph)
for i in concept.values:
    a = matcher.match('股票', TS代码=i[0]).first()
    b = matcher.match('概念', 概念名称=i[1]).first()
    r = Relationship(a, '概念属于', b)
    graph.create(r)

In [29]:
news = pd.read_csv('1news_data_table.csv', encoding='gbk')

news_iden = news[['公告']].drop_duplicates(keep='first', inplace=False)
for i in news_iden.values:
    a = Node('公告', 公告=i[0])
    graph.create(a)

matcher = NodeMatcher(graph)
for i in news.values:
    a = matcher.match('股票', TS代码=i[0]).first()
    b = matcher.match('公告', 公告=i[2]).first()
    r = Relationship(a, '发布公告', b, 时间=i[1])
    graph.create(r)

In [22]:
# graph.delete_all()